In [8]:
import pandas as pd
import requests

def api_bitcointrade(key, link, status, pair_, amount_reg, data_init, code_asset):
    import requests

    #Defina sua chave de API aqui
    api_key = key

    #defina a URL da API
    url = link

    #Adicione sua chave de API às suas configurações de cabeçalho
    headers = {'x-api-key': api_key,'Content-Type': 'application/json'}

    #Defina os dados que deseja enviar à API

    params = {
      "status": {status},
      "pair": {pair_},
      "registers_count": {amount_reg},
      "start_date": {data_init}}


    #Faça a chamada à API
    response = requests.get(url, headers=headers, params=params)

    #Imprima a resposta
    # print(response.json());

    # transforma o response.json em um DataFrame
    df = pd.DataFrame(response.json()['data']['orders'])

    # limpando o dataframe de informações desnecessarias
    df.drop(columns= ['code','update_date','subtype','remaining_amount','remaining_price','requested_amount','status','total_price'], inplace=True)

    # Renomeando as colunas do dataframe
    df.columns=['Data Operação','Quantidade','Código Ativo','Operação C/V','Preço Unitário']

    # criando colunas necessarias e declarando o conteudo delas
    df[['Categoria','Corretora','Corretagem','Taxas','Impostos','IRRF']] = ['Criptomoedas','BitcoinTrade', 0, 0, 0, 0]

    # Altera o formato de ISO 8601 da coluna "data operação" para date = dd/mm/aaaa
    df['Data Operação'] = pd.to_datetime(df['Data Operação'])
    df['Data Operação'] = df['Data Operação'].dt.strftime('%d/%m/%Y')

    # formatando e limpando as informações do dataframe que estavam em ingles e deixando no formato aceito pela bitcointrade
    df['Operação C/V'] = df['Operação C/V'].apply(lambda x: 'Compra' if x == 'buy' else 'Venda')
    df['Preço Unitário'] = df['Preço Unitário'].round(3)
    df['Código Ativo'] = df['Código Ativo'].apply(lambda x : code_asset if x in pair_ else 'NA')

    # rearranjando a ordem das colunas do dataframe
    df = df[['Data Operação','Categoria','Código Ativo','Operação C/V','Quantidade','Preço Unitário',\
                          'Corretora','Corretagem','Taxas','Impostos','IRRF']]

    # exportando o DataFrame pronto para o excel em formato .xlsx
    df.to_excel(f'report_bitcointrade_{code_asset}.xlsx', engine='openpyxl', index=False)

    return df

api_bitcointrade(key='API-KEY',
                 link='https://api.bitcointrade.com.br/v3/market/user_orders/list',
                 status='executed_completely',
                 pair_='BRLETH',
                 amount_reg= 100,
                 data_init='2020-01-01T10:51:24.580Z',
                 code_asset='ETH')